In [1]:
#ignition delay for RMG model of propane 
import pandas as pd
import numpy as np
import time
import cantera as ct
import sys 

In [2]:
print(f"Runnning Cantera version: {ct.__version__}")

#constants
pressure = 101325 #Pascals
mole_frac_dict = {"C3H8(1)": 1, "O2(2)": 5, "Ar":94} #this technically sets equivalence ratio to 1 
#mole_frac_dict_eduardos = {"C3H8(1)": 1, "O2(2)": 6, "Ar":93}

def ignition_delay(states, species): 
    """
    This function computes the ignition delay from the occurence of the peak in species' concentration
    """
    # pd = states.to_pandas()
    # print(pd)
    i_ign = states(species).Y.argmax()
    return states.t[i_ign]

def gas_conditions(temp): 
    """
    define gas conditions
    """
    gas.TPX = temp, pressure, mole_frac_dict

Runnning Cantera version: 2.5.1


In [5]:
#to save data
taus = []


reference_species = "OH(6)"

x_axis_vals= list(np.linspace(0.55, 0.8, 20))


for i in x_axis_vals: 
        
    gas = ct.Solution("./cantera/chem_annotated.yaml")
    temp= 1000/i
    gas_conditions(temp)
    
    # Create a batch reactor object and add it to a reactor network
    # In this example, the batch reactor will be the only reactor in the network
    r = ct.IdealGasReactor(contents=gas, name="Batch Reactor")
    reactor_network = ct.ReactorNet([r])
    
    
    # use the above list to create a DataFrame
    time_history = ct.SolutionArray(gas, extra="t")

    t0 = time.time()
    
    # This is a starting estimate. If you do not get an ignition within this time, increase it
    estimated_ignition_delay_time = 0.4
    t = 0

    counter = 1
    while t < estimated_ignition_delay_time:
        t = reactor_network.step()
        if not counter % 10:
            # We will save only every 10th value. Otherwise, this takes too long
            # Note that the species concentrations are mass fractions
            time_history.append(r.thermo.state, t=t)
        counter += 1

    # We will use the 'OH(6)' species to compute the ignition delay
    tau = ignition_delay(time_history, reference_species)

    # Toc
    t1 = time.time()

    print(f"Computed Ignition Delay: {tau:.3e} seconds. Took {t1-t0:3.2f}s to compute")

    #save to large data array
    taus.append(tau)
    
    # If you want to save all the data - molefractions, temperature, pressure, etc
    #time_history.to_pandas().to_csv("time_history.csv")


Computed Ignition Delay: 6.814e-05 seconds. Took 1.16s to compute
Computed Ignition Delay: 7.240e-05 seconds. Took 0.85s to compute
Computed Ignition Delay: 7.112e-05 seconds. Took 0.88s to compute
Computed Ignition Delay: 7.539e-05 seconds. Took 0.90s to compute
Computed Ignition Delay: 8.432e-05 seconds. Took 0.95s to compute
Computed Ignition Delay: 9.164e-05 seconds. Took 1.05s to compute
Computed Ignition Delay: 1.055e-04 seconds. Took 1.00s to compute
Computed Ignition Delay: 1.249e-04 seconds. Took 1.01s to compute
Computed Ignition Delay: 1.485e-04 seconds. Took 1.02s to compute
Computed Ignition Delay: 1.817e-04 seconds. Took 1.19s to compute
Computed Ignition Delay: 2.233e-04 seconds. Took 1.04s to compute
Computed Ignition Delay: 2.870e-04 seconds. Took 1.35s to compute
Computed Ignition Delay: 3.797e-04 seconds. Took 1.16s to compute
Computed Ignition Delay: 5.055e-04 seconds. Took 1.08s to compute
Computed Ignition Delay: 6.954e-04 seconds. Took 1.24s to compute
Computed I

In [48]:
#save to csv file 
dataframe = pd.DataFrame([x_axis_vals, taus]) 
dataframe.to_csv('C3H8_ignition_delays_eds_stoic.csv', index=False)

In [6]:
#save to csv file 
dataframe = pd.DataFrame([x_axis_vals, taus]) 
dataframe.to_csv('C3H8_ignition_delays.csv', index=False)

In [7]:
nora_df = pd.read_csv('/work/westgroup/nora/Code/projects/PFAS/models/C3H8_CH2F2/C3H8/C3H8_davids_settings_branching/C3H8_ignition_delays.csv')
nora_df.iloc[1].values

array([6.81426807e-05, 7.23993438e-05, 7.11177228e-05, 7.53888263e-05,
       8.43228954e-05, 9.16356488e-05, 1.05537980e-04, 1.24853045e-04,
       1.48472959e-04, 1.81719256e-04, 2.23276796e-04, 2.86987392e-04,
       3.79699288e-04, 5.05536773e-04, 6.95442853e-04, 9.71632618e-04,
       1.36864714e-03, 1.94134963e-03, 2.77099482e-03, 3.96513719e-03])